<a href="https://colab.research.google.com/github/NULabTMN/homework-1-batuhanisik751/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this exercise, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs.  As we discussed in class, this avoids performing cubic-time operations on a graph with a large number of nodes.

Furthermore, you should take advantage of the _sparsity_ of the original hyperlinks, even though "teleportation" or random jumps connect every note to every other node.  As long as the maximum degree of the original link graph is much less than the number of nodes, you should be able to keep each iteration's runtime less than quadratic in the number of nodes.

Consider the following directed graph:

![A directed link graph](https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200//pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [ ]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [ ]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. In most real-world collections of hyperlinks, unlike this example, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm we discussed in class. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [ ]:
import numpy as np
from collections import defaultdict
import math
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.

def page_rank_fixed_iter(nodes, edges, iterations=10):
    # Create a record
    record = {i: [] for i in range(len(nodes))}
    initial = [1/len(nodes)]*len(nodes)

    # Apply the source and destination from the prev defined edges
    for source, dest in edges:
        record[source].append(dest)
    for _ in range(iterations):
        updated = [0] * len(nodes)

        for source in range(len(nodes)):
            # Check for links
            if len(record[source]) > 0:
                # Add link
                for dest in record[source]:
                    updated[dest] += initial[source] / len(record[source])

        for i in range(len(nodes)):
            # Update for the value of every i
            updated[i] = (0.15/len(nodes)) + (0.85) * updated[i]
        initial = updated

    result = []
    for id, name in nodes:
        result.append((id, name, initial[id]))

    return result

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
page_rank_fixed_iter(small_nodes, small_edges, 1)
page_rank_fixed_iter(small_nodes, small_edges, 10)
page_rank_fixed_iter(small_nodes, small_edges, 100)

## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [ ]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [ ]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.
edges = []
nodes = []

with open('edges-edu.txt') as file_e:
    for e in file_e:
        source, dest = e.split()
        edges.append((int(source), int(dest)))

with open('vertices-edu.txt') as file_v:
    for v in file_v:
        id, name = v.split()
        nodes.append((int(id), name))

Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall the definition of entropy from our discussion of data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [ ]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

def perplexity(initial):
    value = 0
    for p in initial:
        if p > 0:
            value -= p * math.log2(p)
        else:
            pass
    value = 2 ** value
    return value

def page_rank(nodes, edges, threshold=1):
    # Create a record
    record = {i: [] for i in range(len(nodes))}
    initial = [1/len(nodes)]*len(nodes)

    # Apply the source and destination from the prev defined edges
    for source, dest in edges:
        record[source].append(dest)

    # Add perplexity
    perp_old = perplexity(initial)
    print(f"At iteration0 the perplexity is = {perp_old}")

    iter = 1
    while True:
        updated = [0]*len(nodes)
        for source in range(len(nodes)):
            # Check for links
            if len(record[source]) > 0:
                # Add link
                for dest in record[source]:
                    updated[dest] += initial[source] / len(record[source])

        for i in range(len(nodes)):
            # Update for the value of every i
            updated[i] = (0.15/len(nodes)) + (0.85) * updated[i]

        initial = updated

        # Add perplexity
        perp_new = perplexity(initial)
        print(f"At iteration{iter} the perplexity is = {perp_new}")

        # Check perplexity, VERY IMPORTANT!
        if abs(perp_new - perp_old) < threshold:
            break

        perp_old = perp_new
        iter += 1

    result = []
    for id, name in nodes:
        result.append((id, name, initial[id]))

    return result

PR = page_rank(nodes, edges, 1)
print(PR)

## Link Analysis

\[20 points\] In this final section, you will compute some properties of this web-site graph.

First, consider the _in-link count_ of a website, simply the number of websites pointing to it (including self-links).

In [ ]:
# TODO: List the document ID, domain name, and in-link count of the 70 websites with the highest in-link count
def extract(a):
    return a[2]

def in_link(nodes, edges):
    # Create a dict
    dictionary = {id: 0 for id, _ in nodes}

    for source, dest in edges:
        dictionary[dest] += 1

    result = []
    for id, name in nodes:
        # Append the id name and link count
        result.append((id, name, dictionary[id]))

    # Sort by descending order
    result.sort(key=extract, reverse=True)

    return result

dictionary = in_link(nodes, edges)[:70]
for id, name, count in dictionary:
    print(f"Document ID: {id}, Domain Name: {name}, In-Link Count: {count}")

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [ ]:
# TODO: List the document ID, domain name, and PageRank of the 70 websites with the highest PageRank.
top_70 = page_rank(nodes, edges, threshold=1)
# Sort for highest PageRank
top_70.sort(key=extract, reverse=True)
for id, name, count in top_70[:70]:
    print(f"Document ID: {id}, Domain Name: {name}, PageRank: {count}")

Finally, compute some summary statistics on this dataset.

In [ ]:
# TODO: Compute:
def proportion_calculator(nodes, edges, pr):
    count_out = {node_id: 0 for node_id, _ in nodes}
    count_in = {node_id: 0 for node_id, _ in nodes}

    # Count the number of source nodes and sink nodes
    for source, dest in edges:
        count_out[source] += 1
        count_in[dest] += 1

    # Source node calculation
    source_n = sum(1 for a in count_in.values() if a == 0) / len(nodes)
    # Sink node calculation
    sink_n = sum(1 for a in count_out.values() if a == 0) / len(nodes)
    # Above uniform calculation
    above_uniform = sum(1 for _, _, a in pr if a > 1 / len(nodes)) / len(nodes)

    return {
        "source_nodes_proportion": source_n,
        "sink_nodes_proportion": sink_n,
        "above_uniform_proportion": above_uniform
    }

proportion = page_rank(nodes, edges, threshold=1)
percentage = proportion_calculator(nodes, edges, proportion)

# - the proportion of websites with no in-links (i.e., source nodes);
print(percentage['source_nodes_proportion'])
# - the proportion of websites with no out-links (i.e., sink nodes);
print(percentage['sink_nodes_proportion'])
# - the proportion of websites whose PageRank is higher than the initial uniform distribution.
print(percentage['above_uniform_proportion'])